In [4]:
from datetime import datetime, timedelta
import pandas as pd

In [ ]:
def detect_weird_timeint(data, time_window):
    '''
    '''
    start_index = 0
    end_index = 1
    
    anomalies = pd.DataFrame(columns=["start_index", "end_index", "start_time", "end_time", "num_adid", "adids"])
    
    while (start_index < len(data) and end_index < len(data)):
        print("-"*50)
        print(start_index, end_index)
        end_index += 1
        print(start_index, end_index)
        
        if (end_index == len(data)) or (data.time[end_index] - data.time[start_index] > time_window):
            
            adids = data.loc[start_index:(end_index-1), "adid"].unique()
            num_adid = len(adids)
            
            if num_adid > 1:
                anomalies.loc[len(anomalies)] = [start_index, end_index-1, data.time[start_index],
                                                data.time[end_index-1], num_adid, list(adids)]
                
                start_index += 1
                
    return anomalies

In [2]:
START_TIME = datetime(2020,1,1)
time_thres = timedelta(minutes=10)
ADIDS = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M"]

In [5]:
testcase1_df = pd.DataFrame(columns=["time", "adid"])

for i in range(5):
    testcase1_df.loc[i,:] = [START_TIME + i*time_thres/10, ADIDS[i]]
    
testcase1_df = testcase1_df.sort_values(by="time")
testcase1_df

,time,adid
0,2020-01-01 00:00:00,A
1,2020-01-01 00:01:00,B
2,2020-01-01 00:02:00,C
3,2020-01-01 00:03:00,D
4,2020-01-01 00:04:00,E


In [ ]:
detect_weird_timeint(testcase1_df, timedelta(minutes=10))